In [1]:
import torch

from packages import *

from BranchAndBound import BranchAndBound
from NeuralNetwork import NeuralNetwork

In [92]:
fileName = "Test3-5-3.pth"
pathToStateDictionary = "Networks/" + fileName
network = NeuralNetwork(pathToStateDictionary)
dim = network.Linear[0].weight.shape[1]

outputDim = network.Linear[-1].weight.shape[0]
device = torch.device("cpu")
network.to(device)

W = []
bias = []
print(network.Linear)
for i in range(len(network.Linear)):
    if i%2:
        continue
    W.append(torch.Tensor(network.Linear[i].weight))
    bias.append(torch.Tensor(network.Linear[i].bias))


W, bias

Sequential(
  (0): Linear(in_features=3, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=3, bias=True)
)


([tensor([[-0.1564, -0.1740, -0.0865],
          [-0.4800, -0.1514, -0.2694],
          [ 0.5363,  0.0138,  1.0077],
          [-0.0228, -0.1505,  0.0774],
          [-0.4973, -0.4906,  0.3249]], grad_fn=<AliasBackward0>),
  tensor([[-0.0616, -0.3559, -1.8353, -0.1595, -0.3297],
          [ 0.1903, -0.0918, -0.2180,  0.2999,  0.0706],
          [ 0.1374, -0.0473, -0.0129,  0.0225,  0.3771]],
         grad_fn=<AliasBackward0>)],
 [tensor([-0.2415, -0.5304, -1.2769, -0.2195, -0.4591], grad_fn=<AliasBackward0>),
  tensor([ 1.2264,  3.1160, -0.8564], grad_fn=<AliasBackward0>)])

In [93]:
A = torch.zeros((2*dim, 2*dim))
A[:dim, dim:] = torch.eye(dim)
A

tensor([[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [94]:
g = 9.8

B = torch.zeros((2*dim, dim))
B[dim:] = torch.eye(dim)
B[3,0] = g
B[4,1] = -g
B

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 9.8000,  0.0000,  0.0000],
        [ 0.0000, -9.8000,  0.0000],
        [ 0.0000,  0.0000,  1.0000]])

In [103]:
class NeuralNetworkforReachSDP(nn.Module):
    def __init__(self, path, A, B):
        self.statedim = 6
        self.controlstate = 3

        super().__init__()
        stateDictionary = torch.load(path, map_location=torch.device("cpu"))
        layers = []
        for keyEntry in stateDictionary:
            if "weight" in keyEntry:
                layers.append(nn.Linear(stateDictionary[keyEntry].shape[1] + self.statedim,
                                             stateDictionary[keyEntry].shape[0] + self.statedim))
                layers.append(nn.ReLU())
        
        layers.append(nn.Linear(self.statedim + self.controlstate, self.statedim))
        
        self.Linear = nn.Sequential(
            *layers
        )

        print(self.Linear[0].weight)
        for i in range(len(network.Linear)):
            if i%2:
                continue
            self.Linear[i].weight[]
        
        # print(self.state_dict)
        # print(self.Linear[0].weight)

    def load(self, path):
        stateDict = torch.load(path, map_location=torch.device("cpu"))
        self.load_state_dict(stateDict)

    # @TODO
    # def train(self):

    def forward(self, x):
        return self.Linear(x)

In [104]:
network = NeuralNetworkforReachSDP(pathToStateDictionary, A, B)


Parameter containing:
tensor([[-0.1638, -0.3185,  0.0115, -0.1761,  0.3117, -0.1502,  0.0601,  0.2689,
         -0.0373],
        [-0.0224, -0.2686,  0.0427, -0.3298,  0.2919, -0.2436, -0.0060, -0.2801,
         -0.2731],
        [ 0.2951, -0.2692, -0.1594,  0.0570, -0.1634, -0.0800,  0.1281, -0.1019,
         -0.0089],
        [ 0.2855, -0.1600,  0.2281, -0.1333, -0.0440,  0.0587, -0.1725, -0.0618,
         -0.3060],
        [ 0.1940,  0.0892,  0.0946,  0.0583,  0.2580,  0.2573, -0.0520, -0.0121,
         -0.0629],
        [ 0.3010, -0.1507, -0.2232,  0.3107, -0.2491, -0.2080, -0.2539,  0.3000,
          0.1587],
        [-0.3294,  0.3243, -0.0822, -0.2960,  0.0641,  0.3128, -0.0441,  0.0621,
          0.0289],
        [-0.0246,  0.3241,  0.2621, -0.2734, -0.2330,  0.1425, -0.0538,  0.0265,
         -0.1396],
        [ 0.0204,  0.2033,  0.2516,  0.2451, -0.1466, -0.1373, -0.2390,  0.3216,
         -0.1084],
        [-0.3175, -0.0475, -0.0063,  0.3017,  0.2401,  0.0745,  0.2706, -0.163